In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

In [2]:
from tensorflow.keras.applications import EfficientNetV2B1
model = EfficientNetV2B1(include_top=False, pooling='avg', weights='imagenet')
model.trainable = False

In [3]:
print(model.get_layer('avg_pool'))

In [4]:
#model.summary()

In [5]:
from keras.models import Model
my_model= Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)

In [6]:
#my_model.summary()

In [7]:
import cv2
import numpy as np
import pandas as pd
import os
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input


In [8]:
image_path = "images_30k"
#image_path = "images_8k"
filenames = os.listdir(image_path)
#print(filenames)
im_size = 240

images = []

   
for i in range(len(filenames)):
    print(i, end="\r")
    #print(image_path + '/' + filenames[i])
    img = cv2.imread(image_path + '/' + filenames[i])
    img = cv2.resize(img, (im_size, im_size))
    images.append(img)  # if memory fails us again try to convert here to float32


In [16]:
images = np.array(images)
print(type(images))
#images = images.astype('float32') / 255.0  # with 30k imageset it causes not enough memory error 
images.shape

<class 'numpy.ndarray'>


(31783, 240, 240, 3)

In [18]:
outputs = []
for i in range(len(images)):
    print(i, end="\r")
    outputs.append(my_model(images[i]))

#pp.pprint(outputs)    

In [19]:
print(np.shape(outputs))

(31783, 1, 1280)


In [20]:
outputs_np = np.array(outputs, dtype="float32")
print(outputs_np.shape)
output_fn = "img_embs_30k.bin"
#output_fn = "img_embs_8k.bin"
outputs_np.tofile(output_fn)

(31783, 1, 1280)


In [21]:
input = np.fromfile(output_fn, dtype='float32')
input = np.reshape(input,(outputs_np.shape))
print(input.shape)

(31783, 1, 1280)


In [22]:
print(np.array_equal(outputs_np,input))

True


#old

img_path='img.jpg'
my_img=imread(img_path)
#imshow(my_img)

SIZE = 224

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h = SIZE # img.shape[0] - h - fixed
w = round(img.shape[1]*(h/img.shape[0])) # keeps the ratio
new_img = cv2.resize(img, (w,h))
w_start=round((w-h)/2)
w_end=w_start+SIZE
crop_img = new_img[0:SIZE, w_start:w_end]
imshow(crop_img)

x = np.expand_dims(crop_img,axis=0)
x = preprocess_input(x)

print('input image shape:', x.shape)

#old

img_path='img.jpg'
my_img=imread(img_path)
imshow(my_img)

#old

outputs = my_model(my_img)

#old

pp.pprint(outputs)